In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")
import re

In [2]:
data = pd.read_excel("../newsdata/NewsResult_20160101-20201231.xlsx")

In [3]:
# 사용할 컬럼 추출
data = data[['일자','특성추출(가중치순 상위 50개)']]

In [4]:
data.dtypes

일자                    int64
특성추출(가중치순 상위 50개)    object
dtype: object

In [5]:
data['일자'] = data['일자'].astype(str)  #데이터타입 string으로 변경 (연도,월 까지 표기하기 위해)

In [6]:
date = []
for x in data['일자']:
    date.append(x[0:6])
data['일자'] = date

In [7]:
data.head()

,일자,특성추출(가중치순 상위 50개)
0,202012,"방사광,청주,부동산,가속기,지원단,충북,청주시,충청북도,차세대,추진지원단,정진규"
1,202012,"후보자,부동산,대구,밀양,박범계,매출액,변호사,경남,법무법인,1천만,국회의원,a씨,..."
2,202012,"부동산,서울,전셋값,팀장,보유세,다주택자,전문가,고준석,강북,안성용,다주택자들,장재..."
3,202012,"수도권,서울,아파트값,상승률,거래량,지점장,부동산원,세종,강남,부동산,6개월,신한은..."
4,202012,"대구,부동산,경남,지명자,밀양,a씨,실거래,관계자,박범계,더불어민주당,14억,끼리,..."


In [8]:
data.info() #null 값이 있는지 체크

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18990 entries, 0 to 18989
Data columns (total 2 columns):
일자                   18990 non-null object
특성추출(가중치순 상위 50개)    18990 non-null object
dtypes: object(2)
memory usage: 296.8+ KB


In [9]:
# 일자별 키워드 합치기
data = data.groupby('일자').sum()
data

,특성추출(가중치순 상위 50개)
일자,
201601,"부동산,주택담보대출,신혼집,산본,전셋값,거래량,부동산시장,반월동,안산,상승폭,관계자..."
201602,"임대주택,뉴스테이,부산도시공사,임대료,부산,lh,입주민들,입주자,공공임대주택,국민임..."
201603,"부동산,2천,전북,봄바람,상승세,하락세,거래량,수도권,한국감정원,활황세,관계자,1천..."
201604,"길음뉴타운,분양가,서울,거주자,성북구,북한산,1순위,롯데캐슬,우이신설선,견본주택,5..."
201605,"충북,충북지역,농산물,비수도,상승세,비수도권,가정경제,1만,소비자물가,주택가격,축산..."
201606,"울산,한국감정원,대구,매매가격,수도권,하락세,동구,피로감,아파트값,근로자,부동산,울..."
201607,"청주,전세난,청주지역,한국감정원,집주인,임대차,부동산,저금리,1만,전월세,입주량서울..."
201608,"서울,부동산,상승률,재건축,강남,중도금,부산,양극화,상승폭,수도권,매매가격,공급과잉..."
201609,"포항,계약금,문장건설,경북,건설사,부동산,경북도청,포항시,지엔하임,6개,동해안발전본..."


In [10]:
# 키워드 value를 한글만 남기고 모두 삭제
def text_cleaning(text):
    text = str(text)
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글의 정규표현식을 나타냅니다.
    result = hangul.sub(' ', text)
    return result

In [11]:
data['특성추출(가중치순 상위 50개)'] = data['특성추출(가중치순 상위 50개)'].apply(lambda x: text_cleaning(x))

In [12]:
from konlpy.tag import Okt
from collections import Counter

# konlpy의 형태소 분석기로 명사 단위의 키워드를 추출합니다.
data['상승'] = 0
data['하락']=0
for i in range (0,len(data['특성추출(가중치순 상위 50개)'])):
    nouns_tagger = Okt()
    nouns = nouns_tagger.nouns(data['특성추출(가중치순 상위 50개)'][i])
    count = Counter(nouns)
    count = dict(count)

    up_key_list = []
    down_key_list = []
    for x in count.keys():
        if '상승' in x:
            up_key_list.append(x)
        if '하락' in x:
            down_key_list.append(x)

    up_count = 0
    down_count = 0
    for x in up_key_list:
        up_count= up_count+ count.get(x)
    for x in down_key_list:    
        down_count= down_count+ count.get(x)
    data['상승'][i] = up_count
    data['하락'][i] = down_count

In [16]:
data = data.drop('특성추출(가중치순 상위 50개)', axis = 1)

In [19]:
data

,상승,하락
일자,,
201601,151,29
201602,57,45
201603,90,35
201604,56,17
201605,72,21
201606,64,28
201607,67,31
201608,59,28
201609,53,24


In [17]:
data.to_csv('20162020.csv')